In [42]:
# Clustering for Vancouver

import pandas as pd
import numpy as np

# load total population of each Vancouver neighbourhood
pop_path = pd.read_csv('CensusLocalAreaProfiles2016.csv' , encoding='latin' )
df_pop = pd.DataFrame(pop_path)
df_pop.head()


,ID,Variable,Arbutus-Ridge,Downtown,Dunbar-Southlands,Fairview,Grandview-Woodland,Hastings-Sunrise,Kensington-Cedar Cottage,Kerrisdale,...,Riley Park,Shaughnessy,South Cambie,Strathcona,Sunset,Victoria-Fraserview,West End,West Point Grey,Vancouver CSD,Vancouver CMA
0,"1, Total - Age groups and average age of the p...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0 to 14 years,2015.0,4000.0,3545.0,2580.0,3210.0,4595.0,7060.0,1880.0,...,3415.0,1175.0,1105.0,1065.0,5460.0,3790.0,1945.0,1900.0,70530.0,362110.0
2,3,0 to 4 years,455.0,2080.0,675.0,1240.0,1320.0,1510.0,2515.0,430.0,...,1175.0,270.0,360.0,360.0,1695.0,1175.0,965.0,420.0,24615.0,116270.0
3,4,5 to 9 years,685.0,1105.0,1225.0,760.0,1025.0,1560.0,2390.0,600.0,...,1160.0,405.0,365.0,365.0,1780.0,1210.0,560.0,670.0,22905.0,122730.0
4,5,10 to 14 years,880.0,810.0,1650.0,580.0,865.0,1525.0,2160.0,845.0,...,1080.0,500.0,375.0,340.0,1985.0,1410.0,415.0,810.0,23010.0,123110.0


In [43]:
# load population of Vancouver neighbourhoods
rest_path = pd.read_csv('Vancouver_restaurant_data.csv', encoding="latin")
df_rest = pd.DataFrame(rest_path)
df_rest.drop(['Rating.1'], axis=1, inplace=True)
df_rest.drop(['number-of-reviews'], axis=1, inplace=True)
df_rest.drop(['veg /Non- Veg'], axis=1, inplace=True)
df_rest.drop(['Rating'], axis=1, inplace=True)
df_rest.drop(['Total No of Ratings'], axis=1, inplace=True)
df_rest.drop(['Current Status'], axis=1, inplace=True)
df_rest.drop(['Opening time'], axis=1, inplace=True)
df_rest.drop(['Dine in availability'], axis=1, inplace=True)
df_rest.drop(['Takeaway type'], axis=1, inplace=True)
df_rest.drop(['Delivery availability'], axis=1, inplace=True)
df_rest.drop(['Description of the Restaurent'], axis=1, inplace=True)
df_rest.drop(['Restaurent Name'], axis=1, inplace=True)
df_rest


,Name of Restaurent,Type of Cusine,Cost,Landmark,Location and Address
0,Las Tortas Mexican,Restaurant,NaN,3353 Cambie Street,NaN
1,The Story Cafe -,Restaurant,NaN,2800 Sexsmith Road,NaN
2,Miku Vancouver,Sushi,Expensive,NaN,200 Granville St # 70 ú In the Granville Square
3,Blue Water Cafe,Seafood,Too Expensive,NaN,1095 Hamilton St ú In Urban Systems
4,Chambar Restaurant,Restaurant,Expensive,NaN,568 Beatty St
...,...,...,...,...,...
379,Shangri-La Hotel Vancouver Drive way,No reviews Hotel,NaN,"Vancouver, BC, Canada",NaN
380,Hilton Vancouver Metrotown,Hotel,NaN,"Burnaby, BC, Canada",NaN
381,Vancouver International Airport,Airport,NaN,"Richmond, BC, Canada",NaN
382,Radisson Hotel Vancouver Airport,Hotel,NaN,"Richmond, BC, Canada",NaN


In [44]:
print(df_rest.Landmark.values)

['3353 Cambie Street' '2800 Sexsmith Road' nan nan nan nan nan nan
 '1038 Canada Pl ú In the Fairmont Pacific Rim' nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan 'Vancouver, BC, Canada ú In Harbour Centre' nan nan nan
 nan 'Vancouver, BC, Canada' nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan 'Vancouver, BC, Canada' nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan
 'Vancouver, BC, Canada ú In the Fairmont Pacific Rim' nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan '646 Kingsway' nan nan
 '545 Granville St ú In the Pacific Centre' '51 W Hastings St' nan nan nan
 nan nan nan nan nan nan nan nan
 'Vancouver, BC, Canada ú In International Village Mall' nan nan n

In [49]:
values = {"Landmark": '', "Location and Address": ''}
df_rest = df_rest.fillna(value = values)

df_rest['Place'] = df_rest['Landmark'] + df_rest['Location and Address']
df_rest.drop(['Landmark'], axis=1, inplace=True)
df_rest.drop(['Location and Address'], axis=1, inplace=True)
print(df_rest.head())


  Name of Restaurent  Type of Cusine           Cost  \
0  Las Tortas Mexican     Restaurant            NaN   
1    The Story Cafe -     Restaurant            NaN   
2      Miku Vancouver          Sushi      Expensive   
3     Blue Water Cafe        Seafood  Too Expensive   
4  Chambar Restaurant     Restaurant      Expensive   

                                             Place  
0                               3353 Cambie Street  
1                               2800 Sexsmith Road  
2  200 Granville St # 70 ú In the Granville Square  
3              1095 Hamilton St ú In Urban Systems  
4                                    568 Beatty St  
